## 42 Usando iluminação e textura em modelos carregados de arquivos OBJ

Modelo obtido no link: https://www.cgtrader.com/free-3d-models/vehicle/sci-fi/saucer-2-and-saucer-2-advanced

In [5]:
import glm
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.PhongShadingShaderProgram_v2 import PhongShadingShaderProgram
from cg.models.ObjLoader_v2 import ObjLoader
from cg.renderers.ModelRenderer_v4 import ModelRenderer
from cg.models.SphereMesh_v2 import SphereMesh
from cg.utils.Texture import Texture

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        self.cameraEye   = glm.vec3(0.0, 0.0, 2.8)
        self.cameraCenter = glm.vec3(0.0, 0.0, -1.0)
        self.cameraUp    = glm.vec3(0.0, 1.0,  0.0)
        self.viewMatrix = glm.lookAt(self.cameraEye, self.cameraCenter, self.cameraUp)

        # carrega objeto
        self.objLoader = ObjLoader('./cg/models/obj/ovni/Saucer_2(adv).obj')
        
        # recupera e imprime a lista de nomes dos itens carregados
        print(self.objLoader.getItemNames())
        
        # cria um renderizador para cada objeto
        self.ovniCoreRenderer = ModelRenderer(self.objLoader.getVertexPositions('Core'),
                                              vertex_normal=self.objLoader.getVertexNormals('Core'),
                                              vertex_tex=self.objLoader.getVertexTextureCoord('Core'))
        
        self.ovniHullRenderer = ModelRenderer(self.objLoader.getVertexPositions('Hull'),
                                              vertex_normal=self.objLoader.getVertexNormals('Hull'),
                                              vertex_tex=self.objLoader.getVertexTextureCoord('Hull'))
        
        # defini posição da luz
        self.lightPosition = glm.vec4(0.0, 0, 2.0, 1)
        
        # cria um shader program 
        self.shaderProgram = PhongShadingShaderProgram()
        
        # ativa o shader programa para configurar uma cor única para todos os vértices
        self.shaderProgram.bind()
        self.shaderProgram.useUniformMaterialColor(True)
        self.shaderProgram.setUniformLightPosition(self.viewMatrix * self.lightPosition)

        # recupera o endereço da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        normal_loc = self.shaderProgram.getVertexNormalLoc()
        texture_loc = self.shaderProgram.getVertexTextureCoordLoc()
        
        # configura os dados dos modelos para serem os dados de entrada do shader program
        self.ovniCoreRenderer.setVertexPositionLoc(position_loc)
        self.ovniHullRenderer.setVertexPositionLoc(position_loc)
        
        # verifica quais atributos de vértices os itens carregados possuem
        if(self.objLoader.hasVertexNormals('Core')):
            self.ovniCoreRenderer.setVertexNormalLoc(normal_loc)
        
        if(self.objLoader.hasVertexNormals('Hull')):
            self.ovniHullRenderer.setVertexNormalLoc(normal_loc)
        
        if(self.objLoader.hasVertexTextureCoord('Hull')):
            self.ovniHullRenderer.setVertexTextureCoordLoc(texture_loc)
    
        # carrega a textura
        self.texture = Texture('./cg/models/obj/ovni/dark-metal-grid-1.jpg')
        
        # inicializa a variável que contém o ângulo de rotação
        self.angle = 0

        # configura a cor de background
        gl.glClearColor(1, 1, 1, 1)
        
        # ativa o teste de profundidade e culling
        gl.glEnable(gl.GL_DEPTH_TEST)
        gl.glEnable(gl.GL_CULL_FACE)
        
    def paintGL(self):
        
        # limpa o background com a cor especificada e o buffer de profundidade
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # calcula as matrizes de transformação e atualiza a matriz do shader
        trans = glm.rotate(glm.mat4(), glm.radians(self.angle), glm.vec3(1.0, 0.0, 0.0))
        scale = glm.scale(glm.mat4(), glm.vec3(0.6, 0.6, 0.6))
        model_matrix = trans * scale
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        self.shaderProgram.setUniformModelViewMatrix(self.viewMatrix * model_matrix)

        # incrementa a variável que contém o ângulo de rotação
        self.angle += 0.01

        # habilita a textura
        self.shaderProgram.bindTexture2D(self.texture.getTextureID())
        
        # renderiza o primeiro item
        self.ovniHullRenderer.render()
        
        # desabilita a textura
        self.shaderProgram.releaseTexture2D()
        
        # mudar a cor no shader e renderiza o segundo item
        self.shaderProgram.setUniformMaterialColor(np.array([0.0, 1.0, 0.0, 1.0], dtype=np.float32))
        self.ovniCoreRenderer.render()
        
        self.update()

    def resizeGL(self, width, height):
        
        # atualiza o viewport
        gl.glViewport(0, 0, width, height)
        
        # atualiza a matriz de projeção perspectiva
        aspectRatio = width / height
        self.perspectiveMatrix = glm.perspective(glm.radians(60.0), aspectRatio, 0.1, 5.0)
        
def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(640, 480)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

material_name = Core - vertex_format = N3F_V3F - min = -2.25019907951355 - max = 2.250200033187866
material_name = Hull - vertex_format = T2F_N3F_V3F - min = -5.9427170753479 - max = 6.9427170753479
['Core', 'Hull']
trying to open ./cg/models/obj/ovni/dark-metal-grid-1.jpg
opened file: size= (2500, 2000) format= JPEG mode= RGB


SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
! jupyter nbconvert --to python 42_Usando_iluminacao_e_textura_em_modelos_carregados_de_arquivos_OBJ.ipynb
%run -i 42_Usando_iluminacao_e_textura_em_modelos_carregados_de_arquivos_OBJ.py

[NbConvertApp] Converting notebook 41_Carregando_modelos_de_arquivos_OBJ_com_iluminacao_e_textura.ipynb to python
[NbConvertApp] Writing 5543 bytes to 41_Carregando_modelos_de_arquivos_OBJ_com_iluminacao_e_textura.py


material_name = Core - vertex_format = N3F_V3F - min = -2.25019907951355 - max = 2.250200033187866
material_name = Hull - vertex_format = T2F_N3F_V3F - min = -5.9427170753479 - max = 6.9427170753479
['Core', 'Hull']
trying to open ./models/obj/ovni/dark-metal-grid-1.jpg
opened file: size= (2500, 2000) format= JPEG mode= RGB
